# 👋🌍 Hello, world: Submit a Q# job to Quantinuum

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Quantinuum](https://www.quantinuum.com/). We will use [Q#](https://learn.microsoft.com/azure/quantum/user-guide/) to express the quantum job.

## Submit a simple job Quantinuum using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example we are using Q#, but note that Qiskit and Cirq are also supported. All code in this example will be written in Python and Q#.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, initialize the `Workspace` as seen below.

In [ ]:
from azure.quantum import Workspace

workspace = Workspace (resource_id = "")

We can use the resulting object to see which _targets_ are available for submission.

In [ ]:
print("This workspace's targets:")
for target in workspace.get_targets():
    print("-", target.name)

### ❕ Do you see `quantinuum.sim.h1-1sc` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Quantinuum to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Quantinuum provider.

## Quantinuum: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Quantinuum's H1-1 device targets.

| Target name | Target ID | Number of qubits | Description|
| --- | ---| ---|---|
H1-1 Syntax Checker | `quantinuum.sim.h1-1sc` | 20 | Quantinuum's H1-1 Syntax Checker. This will return all zeros in place of actual or simulated results. Use this to validate quantum programs against the H1-1 compiler before submitting to hardware or emulators on Quantinuum's platform. Free of cost. |
H2-1 Syntax Checker | `quantinuum.sim.h2-1sc` | 56 | Quantinuum's H2-1 Syntax Checker. This will return all zeros in place of actual or simulated results. Use this to validate quantum programs against the H2-1 compiler before submitting to hardware or emulators on Quantinuum's platform. Free of cost.  |
H1-1 Emulator | `quantinuum.sim.h1-1e` | 20 | Quantinuum's H1-1 Emulator. Uses a realistic physical model and noise model of H1-1. |
H2-1 Emulator | `quantinuum.sim.h2-1e` | 56 | Quantinuum's H2-1 Emulator. Uses a realistic physical model and noise model of H2-1. |
H1-1 | `quantinuum.qpu.h1-1` | 20 | Quantinuum's H1-1 trapped ion device. |
H2-1 | `quantinuum.qpu.h2-1` | 56 | Quantinuum's H2-1 trapped ion device. |

For this example, we will use `quantinuum.sim.h1-1sc` to avoid any costs. If you wish to emulate or run the actual circuit, you may replace all instances of `quantinuum.sim.h1-1sc` in subsequent code cells with one of the other values in the table above, but please note any costs incurred. To learn more about Quantinuum's targets, check out our [documentation](https://aka.ms/AQ/Quantinuum/Documentation).

### 2. Build the quantum program

Let's create a simple Q# program to run.

First, let's initialize the Q# environment and set the target profile to Base Profile. Today, Azure Quantum targets only support the Base Profile, a subset of all Q# commands.

In [ ]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

In [ ]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation GenerateRandomBit() : Result {
    use target = Qubit();

    // Apply an H-gate and measure.
    H(target);
    return M(target);
}

In [ ]:
# Compile the qsharp operation
operation = qsharp.compile("GenerateRandomBit()")

The program you built is a simple quantum random bit generator. With Quantinuum's Syntax Checker, we will be able to confirm that the circuit is able to be run on their H1 emulator and hardware.

### 3. Submit the quantum program to Quantinuum

We will use the `target.submit` function to run the quantum program above on Quantinuum's `quantinuum.sim.h1-1sc` target. This may take a minute or so ⏳. Your job will be packaged and sent to Quantinuum, where it will wait its turn to be run.

In [ ]:
# Set the target to quantinuum.sim.h1-1sc
target = workspace.get_targets("quantinuum.sim.h1-1sc")

# Execute the job. We'll use 100 shots (simulated runs).
job = target.submit(operation, "Generate one random bit", shots=100)
print("Job Id:" + job.id)

result = job.get_results()

The job ID can be used to retrieve the results later using the [get_job method](https://learn.microsoft.com/python/azure-quantum/azure.quantum.workspace?#azure-quantum-workspace-get-job) or by viewing it under the **Job management** section of the portal.

### 4. Visualize the job results
You can view a histogram of the results using [`pyplot`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html):


In [ ]:
from matplotlib import pyplot

pyplot.bar(result.keys(), result.values())
pyplot.title("Result")
pyplot.ylabel("Probability")
pyplot.xlabel("Measurement")

pyplot.show()


**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**

### 5. Next steps
Next, you can try running a program on Quantinuum's emulation target - just replace `quantinuum.sim.h1-1sc` with `quantinuum.sim.h1-1e`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting jobs to Azure Quantum using Q#, refer to [this documentation](https://learn.microsoft.com/azure/quantum/how-to-submit-jobs?pivots=ide-python&tabs=tabid-python).

To learn more about job pricing, review the [Azure Quantum documentation on job costs](https://learn.microsoft.com/azure/quantum/azure-quantum-job-costs).